In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import scipy
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Dropout, Flatten,GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
from sklearn.metrics import classification_report
import numpy as np
from glob import glob

In [2]:
from zipfile import ZipFile
file_name = "chest-ctscan-images.zip"

with ZipFile(file_name,'r') as zip1:
  zip1.extractall()
  print('Done')

Done


In [3]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)
#no flip and zoom for test datase

In [4]:
batch_size = 22

In [5]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('chest-ctscan-images/Data/train',
                                                 target_size = (224, 224),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical')

Found 44 images belonging to 4 classes.


In [6]:
training_set.class_indices

{'adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib': 0,
 'large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa': 1,
 'normal': 2,
 'squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa': 3}

In [7]:
# Make sure you provide the same target size as initialied for the image size
test_set = train_datagen.flow_from_directory('chest-ctscan-images/Data/test',
                                                 target_size = (224, 224),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical')

Found 44 images belonging to 4 classes.


In [8]:
test_set.class_indices

{'adenocarcinoma': 0,
 'large.cell.carcinoma': 1,
 'normal': 2,
 'squamous.cell.carcinoma': 3}

In [9]:
print(test_set.batch_size,test_set.batch_index, sep=', ')

22, 0


In [10]:
# Make sure you provide the same target size as initialied for the image size
valid_set = train_datagen.flow_from_directory('chest-ctscan-images/Data/valid',
                                                 target_size = (224, 224),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical')

Found 44 images belonging to 4 classes.


In [11]:
valid_set.class_indices

{'adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib': 0,
 'large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa': 1,
 'normal': 2,
 'squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa': 3}

In [12]:
from tensorflow.keras.applications import Xception, VGG19, ResNet152V2, InceptionV3, MobileNetV2, DenseNet201, NASNetLarge,  EfficientNetV2L, ConvNeXtTiny


pre_trained_models = [
    Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
    VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
    ResNet152V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
    InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
    MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
    DenseNet201(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
    NASNetLarge(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
    EfficientNetV2L(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
    ConvNeXtTiny(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
]

In [14]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D

classify_models = []
for pretrained_model in pre_trained_models: 
    # The last 15 layers fine tune
    for layer in pretrained_model.layers[:-15]:
        layer.trainable = False

    x = pretrained_model.output
    x = GlobalAveragePooling2D()(x)
    x = Flatten()(x)
    x = Dense(units=512, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(units=512, activation='relu')(x)
    x = Dropout(0.1)(x)
    output  = Dense(units=4, activation='softmax')(x)
    model = Model(pretrained_model.input, output)
    classify_models.append(model)


In [55]:
from tensorflow.python.keras.utils.layer_utils import count_params
import pandas as pd 

result_lst1 = []

for model, pre_model in zip(classify_models, pre_trained_models): 
    result_lst1.append({
        'Model Name': pre_model.name,
        'No of Layers': len(model.layers),
        'No of Conv layers': len([layer for layer in model.layers if 'conv' in layer.name]),
        'Total params': (str(model.count_params()/100000) + 'M'),
        'Trainable params': (str(count_params(model.trainable_weights)/100000) + 'M'),
        'Non-trainable params': (str(count_params(model.non_trainable_weights)/100000) + 'M')
    })
result_df1 = pd.DataFrame.from_dict(result_lst1)
result_df1 = result_df1.set_index('Model Name')
result_df1

,No of Layers,No of Conv layers,Total params,Trainable params,Non-trainable params
Model Name,,,,,
xception,139,111,221.75276M,81.0218M,140.73096M
vgg19,29,16,205.51748M,202.91588M,2.6016M
resnet152v2,571,556,596.45444M,68.33156M,528.12288M
inception_v3,318,94,231.1658M,17.08676M,214.07904M
mobilenetv2_1.00_224,161,5,31.78564M,19.6058M,12.17984M
densenet201,714,692,195.70244M,18.09348M,177.60896M
NASNet,1046,504,872.46422M,23.36324M,849.10098M
efficientnetv2-l,1035,217,1186.67428M,54.7626M,1131.91168M


In [27]:
import tensorflow as tf
import keras.losses

for model in classify_models: 
    optimizer1 = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9)
    model.compile(optimizer=optimizer1, loss=keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [28]:
# fit the model
# Run the cell. It will take some time to execute
model_hist_lst = []
for model, pre_model in zip(classify_models, pre_trained_models): 
  print('Fitting Model: ', pre_model.name)
  hist = model.fit(
    training_set,
    validation_data=test_set,
    epochs=15,
    # steps_per_epoch=len(training_set),
    # validation_steps=len(test_set)
  )
  model_hist_lst.append(hist)
  print()

Fitting Model:  xception
Epoch 1/15


c:\Users\surya\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2/2 ━━━━━━━━━━━━━━━━━━━━ 17s 7s/step - accuracy: 0.2121 - loss: 1.4538 - val_accuracy: 0.3864 - val_loss: 1.4034
Epoch 2/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 5s/step - accuracy: 0.5606 - loss: 1.2525 - val_accuracy: 0.3636 - val_loss: 1.5626
Epoch 3/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 5s/step - accuracy: 0.5303 - loss: 0.9665 - val_accuracy: 0.4318 - val_loss: 2.6415
Epoch 4/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 5s/step - accuracy: 0.6515 - loss: 0.8232 - val_accuracy: 0.3182 - val_loss: 3.7317
Epoch 5/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 8s 5s/step - accuracy: 0.7121 - loss: 0.7669 - val_accuracy: 0.3409 - val_loss: 3.0828
Epoch 6/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 8s 5s/step - accuracy: 0.8485 - loss: 0.4549 - val_accuracy: 0.3636 - val_loss: 3.0254
Epoch 7/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 5s/step - accuracy: 0.9697 - loss: 0.2011 - val_accuracy: 0.3864 - val_loss: 2.8919
Epoch 8/15
2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 5s/step - accuracy: 0.9848 - loss: 0.1410 - val_accuracy: 0.4091 - val_loss: 3.2878
Epoch 9/15
2/2 ━━━━━━━━━━━

ResourceExhaustedError: Graph execution error:

Detected at node StatefulPartitionedCall/gradient_tape/functional_39_1/convnext_xlarge_stage_3_block_2_pointwise_conv_2_1/MatMul/MatMul_1 defined at (most recent call last):
<stack traces unavailable>
OOM when allocating tensor with shape[22,7,8192,2048] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator mklcpu
	 [[{{node StatefulPartitionedCall/gradient_tape/functional_39_1/convnext_xlarge_stage_3_block_2_pointwise_conv_2_1/MatMul/MatMul_1}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_one_step_on_iterator_305005]

In [35]:
# Validation set - True value preparation
pred_set = valid_set
n = len(pred_set)
input_lst = []
output_lst = []
for i in range(n): 
    set1 = next(training_set)
    input1 = set1[0]
    output1 = set1[1]
    y_true1 = np.argmax(output1, axis=1)
    input_lst.append(input1)
    output_lst.extend(y_true1)
input_arr = np.concatenate(input_lst)
output_lst = np.array(output_lst)
print(output_lst)
print(len(output_lst))
print(np.unique(output_lst, return_counts=True))


[3 2 1 0 3 3 3 1 1 3 2 3 2 2 0 2 0 3 0 1 1 3 2 3 0 2 2 2 0 1 0 1 0 0 3 2 1
 3 2 1 1 0 0 1]
44
(array([0, 1, 2, 3], dtype=int64), array([11, 11, 11, 11], dtype=int64))


In [36]:
y_preds = []
for model, pre_model in zip(classify_models, pre_trained_models): 
    predictions = model.predict(input_arr)
    y_pred = np.argmax(predictions, axis=1)
    print('Model Name: ', pre_model.name)
    print('Predictions: ', y_pred)
    print('No of Predictions: ', len(y_pred))
    print('Predictions Class wise: ', np.unique(y_pred, return_counts=True))
    print()
    y_preds.append(y_pred)


2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step
Model Name:  xception
Predictions:  [3 2 1 3 3 3 3 1 3 3 2 3 2 2 0 2 0 3 0 3 1 3 2 3 3 2 2 2 3 1 0 1 3 3 3 2 1
 3 2 3 1 2 3 1]
No of Predictions:  44
Predictions Class wise:  (array([0, 1, 2, 3], dtype=int64), array([ 4,  8, 12, 20], dtype=int64))
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step
Model Name:  vgg19
Predictions:  [2 2 0 2 2 2 2 0 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 0 2 0 2 0]
No of Predictions:  44
Predictions Class wise:  (array([0, 2], dtype=int64), array([ 6, 38], dtype=int64))
1/2 ━━━━━━━━━━━━━━━━━━━━ 9s 10s/stepWARNING:tensorflow:6 out of the last 6 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x0000022070542200> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), 

In [42]:
target_names = [x[0] for x in sorted(pred_set.class_indices.items(), key=lambda x: x[1])]
print(target_names)

['adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib', 'large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa', 'normal', 'squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa']


In [57]:
result_lst2 = []
for model, pre_model, y_pred in zip(classify_models, pre_trained_models, y_preds):
    print('Model Name: ', pre_model.name)
    report = classification_report(output_lst, y_pred, target_names=target_names, zero_division=0.0)
    print(report) 
    print()
    report_dict = classification_report(output_lst, y_pred, target_names=target_names, output_dict=True, zero_division=0.0)
    result_lst2.append({
        'Model Name': pre_model.name,
        'accuracy': report_dict['accuracy'],
        'f1-score': report_dict['weighted avg']['f1-score'],
        'recall': report_dict['weighted avg']['recall'],
        'support': report_dict['weighted avg']['support']
    })
result_df2 = pd.DataFrame.from_dict(result_lst2)
result_df2 = result_df2.set_index('Model Name')
result_df = pd.merge(result_df1, result_df2, on=['Model Name'], how='outer')
result_df = result_df.sort_values(by=['f1-score'], ascending=[False])
result_df

Model Name:  xception
                                                  precision    recall  f1-score   support

      adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib       1.00      0.36      0.53        11
   large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa       1.00      0.73      0.84        11
                                          normal       0.92      1.00      0.96        11
squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa       0.55      1.00      0.71        11

                                        accuracy                           0.77        44
                                       macro avg       0.87      0.77      0.76        44
                                    weighted avg       0.87      0.77      0.76        44


Model Name:  vgg19
                                                  precision    recall  f1-score   support

      adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib       0.17      0.09      0.12        11
   large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa      

,No of Layers,No of Conv layers,Total params,Trainable params,Non-trainable params,accuracy,f1-score,recall,support
Model Name,,,,,,,,,
NASNet,1046,504,872.46422M,23.36324M,849.10098M,1.000000,1.000000,1.000000,44.0
densenet201,714,692,195.70244M,18.09348M,177.60896M,1.000000,1.000000,1.000000,44.0
inception_v3,318,94,231.1658M,17.08676M,214.07904M,1.000000,1.000000,1.000000,44.0
resnet152v2,571,556,596.45444M,68.33156M,528.12288M,0.909091,0.910606,0.909091,44.0
xception,139,111,221.75276M,81.0218M,140.73096M,0.772727,0.760409,0.772727,44.0
mobilenetv2_1.00_224,161,5,31.78564M,19.6058M,12.17984M,0.613636,0.598853,0.613636,44.0
efficientnetv2-l,1035,217,1186.67428M,54.7626M,1131.91168M,0.568182,0.538722,0.568182,44.0
vgg19,29,16,205.51748M,202.91588M,2.6016M,0.272727,0.141657,0.272727,44.0


In [62]:
for model, pre_model in zip(classify_models, pre_trained_models):
    # plot the loss
    plt.plot(model.history['loss'], label='train loss')
    plt.plot(model.history['val_loss'], label='val loss')
    plt.legend()
    plt.savefig(pre_model.name + '_LossVal_loss')
    plt.show()

KeyError: 'loss'